In [1]:
import json
import os
import ast
from tqdm import tqdm
from multiprocessing import Pool
import sys
from random import choice, randint
import time
sys.path.append('/home/sreenath_a/Projects/OSS/Production/GitRipper')

import pandas as pd
from GitRipper.Ripper import collect
from sqlalchemy import create_engine
from sqlalchemy import text as sql_text
from sqlalchemy import func

In [2]:
hostname="oss-1.cij9gk1eehyr.us-east-1.rds.amazonaws.com"
dbname="ossdata"
uname="admin"
pwd="OSS022323"

engine = create_engine(f"mysql+pymysql://{uname}:{pwd}@{hostname}/{dbname}")

In [3]:
columns = ['repo_owner', 'repo_name', 'oid', 'messageHeadline', 'author_name',
       'author_email', 'author_user_login', 'author_user_location',
       'author_user_company', 'author_user_pronouns', 'author_user_bio',
       'author_user_websiteUrl', 'author_user_twitterUsername', 'author_date',
       'additions', 'deletions']

In [4]:
cpu_count = os.cpu_count()
parallelcount = cpu_count
print("CPU Count: ", cpu_count)

CPU Count:  20


In [5]:
#Load the json file
with open('/home/sreenath_a/Projects/OSS/API_keys.json') as f:
    keys = json.load(f)['keys']

In [6]:
keys_list = [(i.split(":")[0], i.split(":")[1]) for i in keys]
collector = collect(keys_list)

In [7]:
statement = "SELECT owner, repo FROM python.package_repo_info LEFT JOIN python.package_commits_info ON owner=repo_owner AND repo=repo_name WHERE repo_owner is NULL AND repo_name is NULL;"

In [8]:
statement_count = "SELECT count(*) FROM python.package_repo_info LEFT JOIN python.package_commits_info ON owner=repo_owner AND repo=repo_name WHERE repo_owner is NULL AND repo_name is NULL;"
with engine.connect() as conn:
    result = conn.execute(sql_text(statement_count))
    row_count = result.scalar()

In [ ]:
row_count

In [ ]:
overall_it = (row_count//parallelcount)+1

In [ ]:
def get_commits(owner, repo, key, since):
    randtime = randint(100, 200)
    time.sleep(randtime/3)
    try:
        commits = collector.collectCommits(owner, repo, key, since)
        return commits
    except Exception as e:
        print("Error: ", e)
        return None

In [ ]:
t = 0
for chunk in tqdm(pd.read_sql_query(con=engine.connect(), sql=sql_text(statement), chunksize=parallelcount), total=overall_it):
    repos = chunk.repo.tolist()
    owners = chunk.owner.tolist()
    best_keys = collector.getBestKeys(n=parallelcount)
    sinces = ["2007-01-01T00:00:00Z" for i in range(parallelcount)]
    args = list(zip(owners, repos, best_keys, sinces))
    with Pool(parallelcount) as p:
        result = p.starmap(get_commits, args)
    result = [i for i in result if i is not None]
    if len(result) == 0:
        continue
    df_result = pd.concat(result)[columns]
    df_result = df_result.reset_index(drop=True)
    if len(df_result) == 0:
        continue
    df_result.to_sql(name='package_commits_info', con=engine, if_exists='append', schema='python', index=False)
    t = t + 1
    if t % 2 == 0:
        time.sleep(60)
    if t % 10 == 0:
        time.sleep(120)